In [ ]:
# 重新建立模型
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,datasets,Sequential
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (Conv2D, Dense, Flatten, Input, Lambda,
                                     MaxPooling2D)

class SIFTnet(keras.Model):
    def __init__(self):
        super(SIFTnet, self).__init__()
        
        self.layer1 = Sequential([layers.experimental.preprocessing.Rescaling(1. / 1),])
            
        self.layer2 = Sequential([layers.Conv2D(64, (3, 3), activation='relu',padding = 'same'),])
        self.layer3 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer4 = Sequential([layers.Conv2D(128, (3, 3), activation='relu',padding = 'same'),])
        self.layer5 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer6 = Sequential([layers.Conv2D(256, (3, 3), activation='relu',padding = 'same'),])
        self.layer7 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer8 = Sequential([layers.Conv2D(512, (3, 3), activation='relu',padding = 'same'),])
        self.layer9 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer10 = Sequential([layers.Flatten(),
#                           layers.Dropout(0.5),
                          layers.Dense(256, activation='relu'),])
        
        self.out1 = Sequential([Dense(256,activation='relu')])
        self.out2 = Sequential([Dense(1,activation='sigmoid')])

        
        
    def call(self, inputs,training = None):
        x = self.layer1(inputs)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        
        y = self.layer1(inputs)
        y = self.layer2(y)
        y = self.layer3(y)
        y = self.layer4(y)
        y = self.layer5(y)
        y = self.layer6(y)
        y = self.layer7(y)
        y = self.layer8(y)
        y = self.layer9(y)
        y = self.layer10(y)
        
        l1_distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([x, y])
        
        out = self.out1(l1_distance)
        out = self.out2(out)
        return x
        
        
# def main():
#     model = SIFTnet(32,0.1,4)
#     model.build(input_shape=(None, 256, 256, 3))
#     model.summary()
    
    
# if __name__ == '__main__':
#     main()



In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image




model= SIFTnet()
model.build(input_shape=(None,256, 256, 3))
model.load_weights('model_data/') # 输入模型
data_Dense = []


np.set_printoptions(linewidth=1000) 

need_GM = 7
storeys = ['9','15','20','25']
groupID_list = ['1','2','3']
ground_list = ['C','D']
          
for storey in storeys:
    for ground in ground_list:
        for groupID in groupID_list:
            picsize =256
            print('组别',storey+ground+groupID)
            # 预测位置
            path = ''+storey+ground+groupID  # 数据集
            print(path)
    
            for path,dirs,files in os.walk(path):
                files_list = files

            flag = 0
            for files_name in files_list:
                file_name = path+os.sep+files_name
                img_init = cv2.imread(file_name)
                img_init = cv2.resize(img_init, (picsize, picsize))
                img = np.asarray(img_init)  
                Dense = model(img.reshape(1,picsize, picsize, 3))
                data_Dense.append(Dense[0].numpy())
                flag=flag+1 
print('完成')

# 将图片输入模型得到向量
np.savetxt('Dense_data\\Sia.txt', data_Dense, fmt="%.4f")

